In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({
    'X': np.array([20, 40, 60, 80]),
    'Y': np.array([60, 40, 22, 22]),
    'Z': np.array([6, 6, 2, 2])})

print(df)

    X   Y  Z
0  20  60  6
1  40  40  6
2  60  22  2
3  80  22  2


In [8]:
dff = df.pivot_table(index = 'X', columns = 'Y', values = 'Z', fill_value = 0)
print(dff)

#df.groupby(['Concentration','Temperature'])['Resistance'].first().unstack()

#df.index.values
#df.columns.values
#df.values

Y   22  40  60
X             
20   0   0   6
40   0   6   0
60   2   0   0
80   2   0   0


In [ ]:
#df['key'] = df.groupby(['Concentration', 'Temperature']).cumcount()
#pp = df.pivot_table(index = ['key', 'Concentration', 'Temperature'], columns = 'Concentration', values = 'Resistance', fill_value = 0).reset_index()
#pp

In [9]:
dic_matrix = pd.Series(df.Z.values, index = [df.X, df.Y]).to_dict()
dic_matrix

{(20, 60): 6, (40, 40): 6, (60, 22): 2, (80, 22): 2}

In [10]:
list = [(k, v) for k, v in dic_matrix.items()]
list

[((20, 60), 6), ((40, 40), 6), ((60, 22), 2), ((80, 22), 2)]

In [ ]:
from typing import Tuple
from IPython import get_ipython
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import copy


def pivoting(df: pd.DataFrame) -> pd.DataFrame:
    """Pivoting function to reshape DataFrame with columns
    ['Temperature', 'Concentration', 'Resistance']
    from long to wide format.
    Args:
        df (pd.DataFrame): Given DataFrame in long format.
    Returns:
        pd.DataFrame: Final DataFrame in wide format, NANs filled with zeros
                      and index sorted.
    """
    df2 = (df.pivot('Temperature', 'Concentration', 'Resistance')
             .reindex(sorted(set(df['Concentration'])), axis=1)
             .fillna(0)
             .astype(int)
             .sort_index(axis=0, ascending=False))
    return df2


def check_matrix(mat: np.array, conc: list, temp: list) -> Tuple[np.array, list, list]:
    """The function checks if the given matrix (nxm) has dimension
    1xm or nx1. If so it adds one column or row filled with zeros,
    respectively. It also make sure that this values are added to
    the concentration and temperature lists.
    Args:
        mat (np.array): Given matrix.
        conc (list): List of given concentrations.
        temp (list): List of given temperatures.
    Returns:
        Tuple[np.array, list, list]: Three variable (final matrix, list of concs, list of temps)
    """
    final_mat = copy.deepcopy(mat)
    if final_mat.shape[0] == 1:
        temp[len(temp)] = 0
        new_mat = np.zeros(mat.shape[1])
        final_mat = np.vstack([final_mat, new_mat])
    if final_mat.shape[1] == 1:
        conc[len(conc)] = 0
        new_mat = np.zeros(final_mat.shape[0]).reshape(-1, 1)
        final_mat = np.hstack([new_mat, final_mat])
    return final_mat, conc, temp


def prepare_matrix(mat: np.array) -> np.array:
    """Function that fills zeros in given matrix if:
    1. There is a higher value (to the right) in the row with zero checked.
    2. There is a higher value (above) in the column with zero checked.
    If both conditions are met, the higher value is chosen.
    Args:
        mat (np.array): Given matrix with zeros.
    Returns:
        np.array: Final matrix with some zeros filled.
    """
    mat_final = copy.deepcopy(mat)
    zero_indices = np.where(mat_final == 0)
    for i, j in zip(zero_indices[0], zero_indices[1]):
        if (max(mat_final[i, j:]) > mat_final[i, j]) and (max(mat_final[i, j:]) > max(mat_final[:i+1, j])):
            mat_final[i, j] = max(mat_final[i, j:])
        elif max(mat_final[:i+1, j]) > mat_final[i, j]:
            mat_final[i, j] = max(mat_final[:i+1, j])
    return mat_final


# %%
get_ipython().run_line_magic('matplotlib', 'inline')

# Plotting function
def plot(mat, conc, temp):
    levels = np.linspace(0, 10)
    img = plt.contourf(mat, extent = [min(conc), max(conc), max(temp), min(temp)], levels = levels, cmap = 'RdYlGn')
    #plt.grid()
    bounds = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    #pylab.colorbar(ticks = bounds)
    plt.colorbar(img, ticks = bounds, label = 'Chemical Resistance')
    #plt.clim(0,10)
    plt.xlabel('Concentration (vol%)', fontsize = 12.0)
    plt.ylabel('Temperature (C)', fontsize = 12.0)


df = pd.read_excel('test_cases.xlsx', sheet_name = 'test1')
n = len(df)
conc = df['Concentration']
temp = df['Temperature']

df2 = pivoting(df)
mat = df2.to_numpy()
mat, conc, temp = check_matrix(mat, conc, temp)
mat = prepare_matrix(mat)
plot(mat, conc, temp)


df = pd.read_csv('test2.csv')
n = len(df)
conc = df['Concentration']
temp = df['Temperature']

df2 = pivoting(df)
mat = df2.to_numpy()
mat, conc, temp = check_matrix(mat, conc, temp)
mat = prepare_matrix(mat)
plot(mat, conc, temp)


df = pd.read_csv('test3.csv')
n = len(df)
conc = df['Concentration']
temp = df['Temperature']

df2 = pivoting(df)
mat = df2.to_numpy()
mat, conc, temp = check_matrix(mat, conc, temp)
mat = prepare_matrix(mat)
plot(mat, conc, temp)

df = pd.read_csv('test4.csv')
n = len(df)
conc = df['Concentration']
temp = df['Temperature']

df2 = pivoting(df)
mat = df2.to_numpy()
mat, conc, temp = check_matrix(mat, conc, temp)
mat = prepare_matrix(mat)
plot(mat, conc, temp)

df = pd.read_csv('test5.csv')
n = len(df)
conc = df['Concentration']
temp = df['Temperature']

df2 = pivoting(df)
mat = df2.to_numpy()
mat, conc, temp = check_matrix(mat, conc, temp)
mat = prepare_matrix(mat)
plot(mat, conc, temp)